# Análisis de los diferentes modelos

## 1. Importación de librerías y modulos

In [1]:
#%pip install -r requirements.txt

In [2]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from module import *

c:\Users\luisb\Documents\GitHub\documentQuestionAnswering\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
emb_model = "baai_small"
llm_model = "llama3"

## 2. Carga de datos 

In [4]:
document_loader = PyPDFDirectoryLoader("./data_testing")
documents = document_loader.load()

In [5]:
chunks = split_documents(documents)

## 3. Creación de la base de datos vectorial

In [6]:
db = Chroma(
    persist_directory="./database_testing", embedding_function=get_embedding_function(emb_model)
)

# Calculate Page IDs.
chunks_with_ids = calculate_chunk_ids(chunks)

# Add or Update the documents.
existing_items = db.get(include=[])  # IDs are always included by default
existing_ids = set(existing_items["ids"])
print(f"Number of existing documents in DB: {len(existing_ids)}")

# Only add documents that don't exist in the DB.
new_chunks = []
for chunk in chunks_with_ids:
    if chunk.metadata["id"] not in existing_ids:
        new_chunks.append(chunk)

if len(new_chunks):
    print(f"👉 Adding new documents: {len(new_chunks)}")
    new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
    db.add_documents(new_chunks, ids=new_chunk_ids)
    db.persist()
    print("Documents added correctly ✅")
else:
    print("✅ No new documents to add")

Number of existing documents in DB: 95
✅ No new documents to add


## 4. Preguntas de interés

Formulamos las preguntas y buscamos en la base de datos los chunks que nos ofrezcan mejor contexto para responderlas.

In [7]:
questions_en = [
    "What is the clause that specifies the conditions for the rescission of the contract in the Arras Contract?",
    "What are the job functions described in the Indefinite Employment Contract?",
    "What penalties are applied if the buyer does not meet the conditions of the Arras Contract?",
]

questions_es = [
    "¿Cuál es la cláusula que especifica las condiciones para la rescisión del contrato en el Contrato de Arras?",
    "¿Cuáles son las funciones laborales descritas en el Contrato de Trabajo Indefinido?",
    "¿Qué sanciones se aplican si la parte compradora no cumple con las condiciones del Contrato de Arras?",
]

questions_cat = [
    "Quina és la clàusula que especifica les condicions per a la rescissió del contracte en el Contracte d'Arres?",
    "Quines són les funcions laborals descrites en el Contracte de Treball Indefinit?",
    "Quines sancions s'apliquen si la part compradora no compleix amb les condicions del Contracte d'Arres?",
]

questions = questions_en + questions_es + questions_cat

In [8]:
contexts = []

for question in questions:
    # Get the top 5 most relevant documents
    results = db.similarity_search_with_score(question, k=5)

    # Make a list of the contexts
    question_contexts = []
    for doc, _score in results:
        question_contexts.append(doc.page_content)

    # Append the context sub-list to the list of contexts
    contexts.append(question_contexts)

print(contexts)

[['Texto Refundido de la Ley del Impuesto sobre Sociedades aprobado por R.D. Legislativo 4/2004 de 5 de marzo.  Señalese en el supuesto de solicitar estos incentivos\n                El trabajador es perceptor de prestaciones por desempleo y ha percibido al menos 3 meses de la prestación contributiva                Que es el primer trabajador/a contratado por la empresa y es un trabajador/a menor de 30 años.\n            FIJO DISCONTINUO                                                                                         2 00        TIEMPO PARCIAL                                 \nCÓDIGO DE CONTRATOFIJO DISCONTINUO\nSIN BONIFICACIÓN\nCON BONIFICACIÓNSIN BONIFICACIÓNTIEMPO COMPLETO\n1 5 01 0 0\nCON BONIFICACIÓNCÓDIGO DE CONTRATO', "Si se reunen los requisitos y condiciones establecidos en la Ley 43/2006 de 29 de diciembre (BOE 30 de diciembre), y pertenece a algún co-lectivo de esta Ley o se cumplen los requisitos y condiciones y pertenece al colectivo mayor de 45 años de la Ley 3/20

Ahora creamos las respuestas esperadas para cada pregunta.

In [9]:
answers_en = [
    "In the Arras Contract, the clause that specifies the conditions for the rescission of the contract is the third clause",
    "The worker will provide services as [job title], included in the professional group of [professional group], for the performance of [job functions] in accordance with the professional classification system in force in the company",
    "The buyer may terminate this contract, agreeing to forfeit the amounts paid as penalty deposits.",
]

answers_es = [
    "En el Contrato de Arras, la cláusula que especifica las condiciones para la rescisión del contrato es la tercera cláusula",
    "El/la trabajador/a prestará sus servicios como [puesto], incluido en el grupo profesional de [grupo profesional], para la realización de las funciones de [funciones laborales] de acuerdo con el sistema de clasificación profesional vigente en la empresa",
    "La parte compradora podrá resolver el presente contrato, aviniéndose a perder las cantidades que ha entregado como arras penitenciales.",
]

answers_cat = [
    "En el Contracte d'Arres, la clàusula que especifica les condicions per a la rescissió del contracte és la tercera clàusula",
    "El/la treballador/a prestarà els seus serveis com [lloc de treball], inclosos en el grup professional de [grup professional], per a la realització de les funcions de [funcions laborals] d'acord amb el sistema de classificació professional vigent a l'empresa",
    "La part compradora podrà resoldre el present contracte, convenint-se a perdre les quantitats que ha entregat com a arres penitencials.",
]

answers = answers_en + answers_es + answers_cat
answers

['In the Arras Contract, the clause that specifies the conditions for the rescission of the contract is the third clause',
 'The worker will provide services as [job title], included in the professional group of [professional group], for the performance of [job functions] in accordance with the professional classification system in force in the company',
 'The buyer may terminate this contract, agreeing to forfeit the amounts paid as penalty deposits.',
 'En el Contrato de Arras, la cláusula que especifica las condiciones para la rescisión del contrato es la tercera cláusula',
 'El/la trabajador/a prestará sus servicios como [puesto], incluido en el grupo profesional de [grupo profesional], para la realización de las funciones de [funciones laborales] de acuerdo con el sistema de clasificación profesional vigente en la empresa',
 'La parte compradora podrá resolver el presente contrato, aviniéndose a perder las cantidades que ha entregado como arras penitenciales.',
 "En el Contracte d

Finalmente creamos el DataFrame para evaluar los modelos.

In [10]:
data_samples = {
    'question': questions,
    'answer': answers,
    'contexts': contexts,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

## 5. Evaluación de los modelos

In [11]:
embeddings = get_embedding_function(emb_model)
llm = Ollama(model=llm_model)

In [15]:
import nest_asyncio

nest_asyncio.apply()

try:
    score = evaluate(
        dataset,
        metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
        llm=llm,
        embeddings=embeddings,
        raise_exceptions=False
    )
except Exception as e:
    print(f"An error ocurred: {e}")

df_score = score.to_pandas()
print(df_score)

Evaluating:  28%|██▊       | 10/36 [01:19<01:31,  3.51s/it]Failed to parse output. Returning None.
Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\luisb\.pyenv\pyenv-win\versions\3.12.2\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\luisb\Documents\GitHub\documentQuestionAnswering\.venv\Lib\site-packages\ragas\metrics\_context_precision.py", line 161, in _ascore
    results = await self.llm.generate(
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\luisb\Documents\GitHub\documentQuestionAnswering\.venv\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\luisb\Documents\GitHub\documentQuestionAnswering\.venv\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  F

                                            question  \
0  What is the clause that specifies the conditio...   
1  What are the job functions described in the In...   
2  What penalties are applied if the buyer does n...   
3  ¿Cuál es la cláusula que especifica las condic...   
4  ¿Cuáles son las funciones laborales descritas ...   
5  ¿Qué sanciones se aplican si la parte comprado...   
6  Quina és la clàusula que especifica les condic...   
7  Quines són les funcions laborals descrites en ...   
8  Quines sancions s'apliquen si la part comprado...   

                                              answer  \
0  In the Arras Contract, the clause that specifi...   
1  The worker will provide services as [job title...   
2  The buyer may terminate this contract, agreein...   
3  En el Contrato de Arras, la cláusula que espec...   
4  El/la trabajador/a prestará sus servicios como...   
5  La parte compradora podrá resolver el presente...   
6  En el Contracte d'Arres, la clàusula que esp